# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [84]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [85]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [86]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 50 #experiment with the batch size

# defining few important variables
image_ht = 120
image_wd = 120
RGB_Channel = 3
img_idx = [0,1,2,4,5,6,8,9,10,12,13,14,16,17,18,20,21,22,24,25,26,28,29] # use this global function in the generator function
num_frame_per_input = len(img_idx)

input_shape = (num_frame_per_input, image_ht, image_wd, RGB_Channel)
number_of_classes = 5 #(thumbs up, thumbs down, move forward, move backward, pause)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [87]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28] #create a list of image numbers you want to use for a particular video
    x = len(img_idx)
    y = image_ht
    z = image_wd
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    target_size = (y,z)
                    resized_image = resize(image, target_size, anti_aliasing=True) #resize function will also normalize the image
                    batch_data[folder,idx,:,:,0] = resized_image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resized_image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resized_image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        
        remaining_folders = len(folder_list)%batch_size
        batch = batch + 1 # last batch
        batch_data = np.zeros((remaining_folders,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((remaining_folders,5)) # batch_labels is the one hot representation of the output
        for folder in range(remaining_folders): # iterating over the remaining folders
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes
                target_size = (y,z)
                resized_image = resize(image, target_size, anti_aliasing=True) #resize function will also normalize the image

                batch_data[folder,idx,:,:,0] = resized_image[:,:,0] #normalise and feed in the image
                batch_data[folder,idx,:,:,1] = resized_image[:,:,1] #normalise and feed in the image
                batch_data[folder,idx,:,:,2] = resized_image[:,:,2] #normalise and feed in the image

            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [88]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [89]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras import layers, models

#write your model here


model = models.Sequential(
[
    # 1st Conv3D layer
    layers.Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    # 2nd Conv3D layer
    layers.Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu'),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
     # 3rd Conv3D layer
    layers.Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu'),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    # Flattening Layer
    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),

    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),

    # Output Layer
    layers.Dense(number_of_classes, activation='softmax')
    
    
])



Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [90]:
optimiser = 'adam' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_37 (Conv3D)          (None, 21, 118, 118, 32)  2624      
                                                                 
 max_pooling3d_37 (MaxPoolin  (None, 10, 59, 59, 32)   0         
 g3D)                                                            
                                                                 
 batch_normalization_37 (Bat  (None, 10, 59, 59, 32)   128       
 chNormalization)                                                
                                                                 
 dropout_66 (Dropout)        (None, 10, 59, 59, 32)    0         
                                                                 
 conv3d_38 (Conv3D)          (None, 8, 57, 57, 64)     55360     
                                                                 
 max_pooling3d_38 (MaxPoolin  (None, 4, 28, 28, 64)   

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [91]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [92]:
model_name = 'model_1' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

# write the REducelronplateau code here
LR = ReduceLROnPlateau(
    monitor='val_loss',    # Metric to be monitored
    factor=0.5,            # Factor by which the learning rate will be reduced
    patience=3,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6,           # Lower bound on the learning rate
    verbose=1              # Print a message when the learning rate is reduced
)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [93]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [94]:
model.fit(train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs=num_epochs, 
          verbose=1,
          #callbacks=callbacks_list,
          validation_data=val_generator,
          validation_steps=validation_steps,
          class_weight=None,
          workers=1,
          initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 50
Epoch 1/30
14/14 [==============================] - 109s 8s/step - loss: 3.1920 - categorical_accuracy: 0.2866 - val_loss: 45.1467 - val_categorical_accuracy: 0.2300
Epoch 2/30
14/14 [==============================] - 106s 8s/step - loss: 2.5284 - categorical_accuracy: 0.3107 - val_loss: 57.5383 - val_categorical_accuracy: 0.2300
Epoch 3/30
14/14 [==============================] - 108s 8s/step - loss: 2.1346 - categorical_accuracy: 0.3937 - val_loss: 30.9235 - val_categorical_accuracy: 0.2300
Epoch 4/30
14/14 [==============================] - 107s 8s/step - loss: 1.9580 - categorical_accuracy: 0.3771 - val_loss: 4.2777 - val_categorical_accuracy: 0.2400
Epoch 5/30
14/14 [==============================] - 106s 8s/step - loss: 1.8572 - categorical_accuracy: 0.4178 - val_loss: 9.6730 - val_categorical_accuracy: 0.3100
Epoch 6/30
14/14 [==============================] - 106s 8s/step - loss: 1.6839 - categorical_accuracy: 0

In [95]:
model.save('model_6.h5')